In [ ]:
import pymumble_py3
import numpy as np
import resampy
import time
from voice_forge import PiperTts
from pymumble_py3.constants import PYMUMBLE_SAMPLERATE
from assistant.components.audio import enrich_with_silence
from assistant.config import (
    MUMBLE_SERVER_PORT,
    MUMBLE_SERVER_PASSWORD,
    PIPER_MODELS_LOCATION
)

In [ ]:
tts = PiperTts("en_US-kathleen-low", PIPER_MODELS_LOCATION)
mumble = pymumble_py3.Mumble(
    host="127.0.0.1",
    user="Query",
    port=MUMBLE_SERVER_PORT,
    password=MUMBLE_SERVER_PASSWORD,
)

mumble.start()

In [ ]:
def synthesize_and_stream(mumble: pymumble_py3.Mumble, tts: PiperTts, text: str):
    data, samplerate = tts.synthesize_stream(text)
    resampled_data = resampy.resample(data, samplerate, PYMUMBLE_SAMPLERATE)

    audio_with_silence = enrich_with_silence(resampled_data, PYMUMBLE_SAMPLERATE, 0.2, 0.5)

    # Send the data to the sound output
    mumble.sound_output.add_sound(audio_with_silence.astype(np.int16).tobytes())

In [ ]:
synthesize_and_stream(mumble, tts, "Why is the sky blue?")

In [ ]:
synthesize_and_stream(mumble, tts, "What is COVID-19?")

In [ ]:
synthesize_and_stream(mumble, tts, "Wait a sec.")

In [ ]:
synthesize_and_stream(mumble, tts, "What happend?")

In [ ]:
synthesize_and_stream(mumble, tts, "Are you still there?")

In [ ]:
synthesize_and_stream(mumble, tts, "Hello?")

In [ ]:
synthesize_and_stream(mumble, tts, "What happend before of before?")

In [ ]:
synthesize_and_stream(mumble, tts, "I'm a cat!")

In [ ]:
synthesize_and_stream(mumble, tts, "I need food! There no more humans on planet...")

In [ ]:
synthesize_and_stream(mumble, tts, "No! I'm a cat! I need food! There no more humans on planet...")

In [ ]:
synthesize_and_stream(mumble, tts, "What is the funniest meme for last 10 years?")

In [ ]:
synthesize_and_stream(mumble, tts, "What is your favorite?")

In [ ]:
synthesize_and_stream(mumble, tts, "Remember a number 32.")

In [ ]:
synthesize_and_stream(mumble, tts, "What is the number I asked to remember?")

In [ ]:
synthesize_and_stream(mumble, tts, "Who are you?")

In [ ]:
synthesize_and_stream(mumble, tts, "What is you purpose?")

In [ ]:
synthesize_and_stream(mumble, tts, "How are you working?")